# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database ERD

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database `data.sqlite`.

In [2]:
import pandas as pd
import sqlite3

In [3]:
conn = sqlite3.connect('data.sqlite')

## Write an Equivalent Query using a Subquery

The following query works using a `JOIN`. Rewrite it so that it uses a subquery instead.

```
SELECT
    customerNumber,
    contactLastName,
    contactFirstName
FROM customers
JOIN orders 
    USING(customerNumber)
WHERE orderDate = '2003-01-31'
;
```

In [4]:
pd.read_sql( """
SELECT
    customerNumber,
    contactLastName,
    contactFirstName
FROM customers
JOIN orders 
    USING(customerNumber)
WHERE orderDate = '2003-01-31'
;
            
""", conn)

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


In [5]:
pd.read_sql( """
SELECT customerNumber, contactLastName, contactFirstName
    FROM customers
        WHERE customerNumber IN (
            SELECT customerNumber
            FROM orders
            WHERE orderDate = '2003-01-31'
            );
            
""", conn)

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


In [6]:
pd.read_sql( """
SELECT customerNumber, contactLastName, contactFirstName
    FROM customers
             """,conn)

,customerNumber,contactLastName,contactFirstName
0,103,Schmitt,Carine
1,112,King,Jean
2,114,Ferguson,Peter
3,119,Labrune,Janine
4,121,Bergulfsen,Jonas
...,...,...,...
117,486,Salazar,Rosa
118,487,Taylor,Sue
119,489,Smith,Thomas
120,495,Franco,Valarie


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [7]:
pd.read_sql( """
SELECT productName, quantityOrdered
    FROM orders o
        JOIN ( 
            SELECT p.productName, od.quantityOrdered
                FROM orderdetails od
            JOIN products p
            USING(productCode)
            ) 
 """, conn)

,productName,quantityOrdered
0,1917 Grand Touring Sedan,30
1,1911 Ford Town Car,50
2,1932 Alfa Romeo 8C2300 Spider Sport,22
3,1936 Mercedes Benz 500k Roadster,49
4,1932 Model A Ford J-Coupe,25
...,...,...
976691,1962 Volkswagen Microbus,49
976692,1958 Chevy Corvette Limited Edition,31
976693,1980’s GM Manhattan Express,41
976694,1954 Greyhound Scenicruiser,11


In [8]:
pd.read_sql( """
SELECT *
    FROM (
            SELECT *
            FROM orderdetails 
            JOIN products 
                USING(productCode)
            GROUP BY productName
            ) AS product_sales
            
""", conn)

,orderNumber,productCode,quantityOrdered,priceEach,orderLineNumber,productName,productLine,productScale,productVendor,productDescription,quantityInStock,buyPrice,MSRP
0,10103,S18_3136,25,86.92,13,18th Century Vintage Horse Carriage,Vintage Cars,1:18,Red Start Diecast,Hand crafted diecast-like metal horse carriage...,5992,60.74,104.72
1,10105,S24_2011,43,117.97,9,18th century schooner,Ships,1:24,Carousel DieCast Legends,All wood with canvas sails. Many extras includ...,1898,82.34,122.89
2,10106,S24_2841,49,65.77,13,1900s Vintage Bi-Plane,Planes,1:24,Autoart Studio Design,Hand crafted diecast-like metal bi-plane is re...,5942,34.25,68.51
3,10106,S24_4278,26,71.00,3,1900s Vintage Tri-Plane,Planes,1:24,Unimax Art Galleries,Hand crafted diecast-like metal Triplane is Re...,2756,36.23,72.45
4,10105,S18_3140,22,136.59,11,1903 Ford Model A,Vintage Cars,1:18,Unimax Art Galleries,"Features opening trunk, working steering system",3913,68.30,136.59
...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,10105,S700_1938,29,86.61,12,The Mayflower,Ships,1:700,Studio M Art Models,Measures 31 1/2 inches Long x 25 1/2 inches Hi...,737,43.30,86.61
105,10105,S700_3962,22,99.31,7,The Queen Mary,Ships,1:700,Welly Diecast Productions,Exact replica. Wood and Metal. Many extras inc...,5088,53.63,99.31
106,10105,S700_1138,41,54.00,5,The Schooner Bluenose,Ships,1:700,Autoart Studio Design,All wood with canvas sails. Measures 31 1/2 in...,1897,34.00,66.67
107,10105,S700_3505,39,92.16,6,The Titanic,Ships,1:700,Carousel DieCast Legends,"Completed model measures 19 1/2 inches long, 9...",1956,51.09,100.17


In [9]:
conn.close()

## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

### A quick note on the SQL  `SELECT DISTINCT` statement:

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [ ]:
# Your code here
# Hint: because one of the tables we'll be joining has duplicate customer numbers, you should use DISTINCT

## Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products That Have Been Ordered by Fewer Than 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [ ]:
# Your code here

## Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit Over 15K

In [ ]:
# Your code here

## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!